# Single Path One-Shot Neural Architecture Search using Random Search

### Make everything a bit faster

Your task is to implement SPOS + Random Search to find the optimal ResNet-18-like architecture
for image classification on CIFAR-10 dataset.

### Proposed search space
The same search space as in the lecture, but without kernel size search.
This means that each search block should only have 3 operations.

### Latency awareness
While searching, you should implement hard constraint on model latency. As a proxy for
latency, use the number of MACs. The baseline ResNet-18 has 37M MACs for images of size 32x32.
The final selected architecture should have at most 30M MACs.
You can find an example computation for the number of MACs in this notebook.

### How to complete the task
Plan for your experiments:
1. Train baseline ResNet-18 model.  **(2 pts)**
2. Finalize supernet code. **(15 pts)**
3. Train supernet with the same hyperparameters except the number of epochs. You should increase
   it by a factor of 3-6. Save supernet weights.  **(10 pts)**
4. Write implementation for Random Search.  **(10 pts)**
5. Run it for 100-1000 iterations by measuring accuracies of models from the supernet.
   Keep track of each model accuracy and latency.  **(5 pts)**
6. Train the best architecture from scratch. For this, you can build the whole supernet, and
   select this architecture for all the forward passes of the training.  **(6 pts)**
7. Compare with the model from step 1. Does your model have a better accuracy?  **(2 pts)**

### Code structure

Main.ipynb - Learn your neural networks here.

resnet.py - ResNet-18 implementation from torchvision, simplified for this project.

supernet.py - Unfinished Supernet implementation.

cifar10.py - Transforms for CIFAR-10 training.


## Setup

Before you begin, make sure to install torch, torchvision, numpy, thop and tqdm libraries.

In [ ]:
!pip install torch torchvision numpy thop tqdm

### Imports

In [ ]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from thop import profile
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_train_transform, get_val_transform
from resnet import resnet18

### Make everything a bit faster

In [ ]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Build datasets and dataloaders for CIFAR-10

In [ ]:
# Change this value if needed.
batch_size = 512

In [ ]:
train_transform = get_train_transform()
val_transform = get_val_transform()

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform,
)
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)
test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

## Create your baseline model

In [ ]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')

In [ ]:
model = resnet18(num_classes=10, zero_init_residual=True)
model.to(device=device)

### Compute the number of MACs and parameters for the model

In [ ]:
macs, params = profile(model, inputs=(torch.zeros(1, 3, 32, 32, device=device),))
print(f'Number of macs: {macs / 1e6:.2f}M, number of parameters: {params / 1e6:.2f}M')

## Train baseline model

### Define loss function

In [ ]:
criterion = nn.CrossEntropyLoss()

### Select hyperparameters

In [ ]:
lr = 0.25
weight_decay = 5e-4
momentum = 0.9
n_epochs = 20  # Longer training gives better results, but let's keep baseline model epochs to 20.

### Build optimizer and scheduler

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * n_epochs)

### Define training and evaluation functions

In [ ]:
def train_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        optimizer: optim.Optimizer,
        scheduler,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.train()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()

        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        wrapped_dataloader.set_description(
            f'(train) Epoch={epoch}, lr={scheduler.get_last_lr()[0]:.4f} loss={total_loss / (i + 1):.3f}'
        )

    return total_loss / len(dataloader), total_correct / total_samples


@torch.no_grad()
def validate_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.eval()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        logits = model(inputs)
        loss = criterion(logits, labels)
        _, predicted_labels = torch.max(logits, 1)
        total_loss += loss.item()
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.shape[0]

        wrapped_dataloader.set_description(f'(val) Epoch={epoch}, loss={total_loss / (i + 1):.3f}')

    return total_loss / len(dataloader), total_correct / total_samples

### Run training

In [ ]:
for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = train_one_epoch(model, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(model, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

### Save trained model weights

In [ ]:
torch.save(model.state_dict(), 'baseline_model.pth')

## Neural Architecture Search - Supernet training

### Create supernet

**Before running code in this section, you need to finish supernet implementation.**

Please, go to `supernet.py` file and inspect the current implementation of SearchBlock and Supernet classes.
Pay attention to the TODOs. You need to implement all of them.

Supernet and BasicBlock classes are modified versions of ResNet and BasicBlock classes from `resnet.py`.

    Tip: to understand how the Supernet is constructed, compare the implementation of Supernet and ResNet classes. You should probably use diff tool in your IDE or something.

Task: briefly describe the differences made to construct supernet.

In [ ]:
from supernet import supernet18

In [ ]:
# TODO: Define inner channel multipliers as in lecture.
channel_multipliers = ...

In [ ]:
supernet = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
supernet.to(device=device)

### Select hyperparameters

In [ ]:
# TODO: define hyperparameters for supernet training.

### Build optimizer and scheduler

In [ ]:
# TODO: build optimizer and scheduler for supernet training.

### Define training function

In [ ]:
# TODO: Copy `train_one_epoch` function here, rename to `pretrain_one_epoch`.
#       Call `model.sample_random_architecture()` before making forward pass on each batch.

In [ ]:
for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = pretrain_one_epoch(supernet, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(supernet, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

### Save supernet weights

In [ ]:
torch.save(supernet.state_dict(), 'supernet.pth')

## Neural Architecture Search - Random Search.

In [ ]:
def random_search(
        trained_supernet: nn.Module,
        train_dataloader: DataLoader,
        val_dataloader: DataLoader,
        device: torch.device,
        n_architectures_to_test: int,
        target_latency: float,
) -> Union[float, List[float]]:
    # TODO: Implement Random search.
    # This function should evaluate `n_architectures_to_test` architectures
    # using the `trained_supernet` on the whole validation dataset. The resulting
    # architecture should have latency not greater than `target_latency`.
    # Rank architectures by validation accuracy.
    pass

In [ ]:
n_architectures_to_test = 100
target_latency = 30 * 1e6

accuracy, best_architecture = random_search(
    supernet, train_dataloader, test_dataloader, device, n_architectures_to_test, target_latency,
)
print(f'best architecture: {best_architecture} (test_accuracy={accuracy:.3%})')

## Train found architecture from scratch

Go to `Create your baseline model` section.

Change model definition to this to build unitialized best architecture from the search space:

```python
model = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
model.to(device=device)
model.sample(best_architecture)
```

After this, simply run all the cells in that section.